<a href="https://colab.research.google.com/github/Hiromi06/machine-translation/blob/main/BERT_all_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [3]:
def load_data(file_path):
  with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
  return [line.strip() for line in lines]

en_train_path = '/content/drive/MyDrive/machine_learning/JP_en_train.txt'
en_test_path = '/content/drive/MyDrive/machine_learning/JP_en_test.txt'
ja_train_path = '/content/drive/MyDrive/machine_learning/JP_ja_train.txt'
ja_test_path = '/content/drive/MyDrive/machine_learning/JP_ja_test.txt'

en_train = load_data(en_train_path)
en_test = load_data(en_test_path)
ja_train = load_data(ja_train_path)
ja_test = load_data(ja_test_path)

#print(en_train[:6])
print(len(en_train))

7887595


In [4]:
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, AdamW, get_linear_schedule_with_warmup

model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name)
bert_masked_model = BertForMaskedLM.from_pretrained(model_name, config=config).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
def get_max_length_batch(text_list, tokenizer, batch_size=32):
    max_length = 0
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        encoded_batch = tokenizer(batch, add_special_tokens=True, padding=False, truncation=False)
        batch_max_length = max(len(encoded_text) for encoded_text in encoded_batch['input_ids'])
        max_length = max(max_length, batch_max_length)
    return max_length

en_train_max_length = get_max_length_batch(en_train, tokenizer)
en_test_max_length = get_max_length_batch(en_test, tokenizer)
ja_train_max_length = get_max_length_batch(ja_train, tokenizer)
ja_test_max_length = get_max_length_batch(ja_test, tokenizer)

max_length = max(en_train_max_length, en_test_max_length, ja_train_max_length, ja_test_max_length)
print(f"Optimal max_length: {max_length}")

KeyboardInterrupt: 

In [9]:
def batch_encode(text_list, tokenizer, max_length, batch_size=32):
    encoded_batches = []
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        encoded_batch = tokenizer(batch, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
        encoded_batches.append(encoded_batch.to(device))
    return encoded_batches

In [10]:
en_train_encoding = batch_encode(en_train, tokenizer, max_length)
en_test_encoding = batch_encode(en_test, tokenizer, max_length)
ja_train_encoding = batch_encode(ja_train, tokenizer, max_length)
ja_test_encoding = batch_encode(ja_test, tokenizer, max_length)

# use torch.nn.DataParallel or torch.nn.parallel.DistributedDataParallel ?

KeyboardInterrupt: 

In [ ]:
# concatenate encoded batches

def concatenate_batches(encoded_batches):
    input_ids = torch.cat([batch['input_ids'] for batch in encoded_batches])
    attention_masks = torch.cat([batch['attention_mask'] for batch in encoded_batches])
    return input_ids, attention_masks

train_input_ids, train_attention_masks = concatenate_batches(en_train_encoding)
val_input_ids, val_attention_masks = concatenate_batches(en_test_encoding)
train_labels, _ = concatenate_batches(ja_train_encoding)
val_labels, _ = concatenate_batches(ja_test_encoding)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# create TensorDataset
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

# create DataLoader
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# check DataLoader
for batch in train_dataloader:
    input_ids, attention_masks, labels = batch
    print("Input IDs:", input_ids)
    print("Attention Masks:", attention_masks)
    print("Labels:", labels)
    break  # 最初のバッチのみ表示

In [ ]:
from torch.cuda.amp import autocast, GradScaler

num_epochs = 3

optimizer = torch.optim.AdamW(bert_masked_model.parameters(), lr=5e-5)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
scaler = torch.cuda.amp.GradScaler()

In [ ]:
from tqdm import tqdm

for epoch in range(num_epochs):
    bert_masked_model.train()
    total_loss = 0

    loop= tqdm(train_dataloader, leave=True)
    for batch in loop:
        input_ids, attention_masks, labels = batch
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

        optimizer.zero_grad()

        # apply AMP (Mixed-Precision Training)
        with torch.cuda.amp.autocast():
          outputs = bert_masked_model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
          loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

        # represent progress and average loss of each epoch
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=total_loss/len(train_dataloader))

        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss}")

    bert_masked_model.eval()
    total_eval_loss = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_masks, labels = batch
            input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

            outputs = bert_masked_model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(val_dataloader)
    print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss}")

In [ ]:
model_save_path = '/content/drive/MyDrive/machine_learning/bert_masked_model_allData.pth'
torch.save(bert_masked_model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
model_load_path = '/content/drive/MyDrive/machine_learning/bert_masked_model_allData.pth'
loaded_model = BertForMaskedLM.from_pretrained(model_name, config=config).to(device)
loaded_model.load_state_dict(torch.load(model_load_path, map_location=device))
print(f"Model loaded from {model_load_path}")


In [ ]:
sample_texts = ["This is a sample sentence for translation."]
encoded_input = tokenizer(sample_texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt").to(device)

loaded_model.eval()
with torch.no_grad():
    outputs = loaded_model(**encoded_input)
    predictions = outputs.logits.argmax(dim=-1)

decoded_output = tokenizer.batch_decode(predictions, skip_special_tokens=True)
for i, text in enumerate(sample_texts):
    print(f"Original: {text}")
    print(f"Translated: {decoded_output[i]}")